# Proboscidea Volcanium

In [1]:
import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

from day16 import *
import networkx as nx
import re
from itertools import product, permutations
from functools import reduce
%matplotlib inline

import pandas as pd

from pyvis.network import Network

TEST_PATH = '../tests/16.txt'
INPUT_PATH = '../input/16.txt'

def nx_to_network(G, **kwargs):
    """To visualize a graph"""
    network = Network(notebook=True, cdn_resources='in_line', **kwargs)
    network.from_nx(G)
    return network

## Part 1: Maximum pressure graph problem

### 1. Read input as a directed graph

For example, suppose you have the following scan output:

```
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
```

We can model this as a graph $G$ with vertices (valves) $V$ and edges (tunnels) $E$.
Each node $v$ has an attribute (flow rate) $f_v$.
The edge weight (time to move between tunnels) is 1.

In [28]:
def read_input(fp):
    raw = open(fp).read()
    lines = raw.split('\n')

    G = nx.DiGraph()

    vertices = re.findall('Valve ([A-Z]{2}).*flow rate=(\d+)', raw)
    flows = [int(flow) for v, flow in vertices]
    valves = [v for v, flow in vertices]
    tunnels = []
    for v, l in zip(valves, lines):
        nodes = re.findall('[A-Z]{2}', l)[1:]
        tunnels.extend([(v, nodes[i]) for i in range(len(nodes))])

    G.add_nodes_from([(v, {"flow_rate": f, 
                           "label": f"{v}: {f}"}) for v, f in zip(valves, flows)])
    G.add_edges_from(tunnels, weight=1)
    return G


G = read_input(TEST_PATH)
network = nx_to_network(G, directed=True)
network.show("test_input.html")

### 2. Transform Graph
We want to transform `G` to an easier graph structure, i.e. one where the time spent at a valve to open it is modeled.
To do this, we need to add extra nodes $v' \in V': \forall v \in V: f_v > 0$.

Functions used from networkx:
- nx.add_nodes_from
- nx.add_edges_from
- nx.set_node_attributes

In [29]:
def update_valve_flow(G, valve, flow):
    G.nodes[valve]['flow_rate'] = flow
    G.nodes[valve]['label'] = f"{valve}: {flow}"

    
def transform(G, start=None):
    G_tf = G.copy()
    valves = filter(lambda valve: G.nodes[valve]['flow_rate'] > 0, G.nodes)
    for v in valves:
        f = G.nodes[v]['flow_rate']
        G_tf.add_node(f"{v}o", flow_rate=f, label=f"{v}o: {f}")
        for n in G.neighbors(v):
            G_tf.add_edge(f"{v}o", n, weight=1)
        G_tf.add_edge(v, f"{v}o", weight=1)
        update_valve_flow(G_tf, v, 0)
    if start is not None:
        G_tf.add_node("START", flow_rate=0, label="START")
        G_tf.add_edge("START", start)
    
    return G_tf

G_tf = transform(G, start='AA')
network = nx_to_network(G_tf, directed=True)
network.show("test_input_transformed.html")

### 3. Calculating the pressure over a path

We're interested in 3 quantities when traversing through the tunnels on our transformed graph `G`:
- suppose a path $P=(v_1, v_2, ..., v_T)$ leads from node $v_1$ to $v_T$. $v_1$ is from rest
- the additional flow gained after traversing the path: $F(p) = \sum_{t=2}^{T} f_{v_t}$
- the pressure released: $\rho(p) = \sum_{t=1}^{T}t f_{v_t}$
- the time (in minutes) it took: $T$

In [30]:
path = ['AA', 'DD', 'DDo', 'CC', 'BB', 'BBo']

def calculate_pressure(G, p, memory=None):
    if memory is not None:
        key = (p[0], p[-1])
        if key in memory:
            return memory[key]
    flow_rates = np.array([G.nodes[valve]['flow_rate'] for valve in p])
    T = len(flow_rates) - 1
    # /!\ Don't include first state!
    P = np.sum(np.arange(T, 0, -1) * flow_rates[1:])
    F = flow_rates[1:].sum()
    if memory is not None:
        memory[key] = P, F, T
    return P, F, T

M = {}
calculate_pressure(G_tf, path, memory=M)
# G_tf.nodes

(93, 33, 5)

In [31]:
M

{('AA', 'BBo'): (93, 33, 5)}

### 4. Distance matrix
Store the shortest distance between all nodes with a flow rate as a matrix.
This can be handy, since we will want to limit the breadth of our search space during our recursion to the K closest open valves of the node under investigation.

**Important observations**
- The shortest path from one open valve to the other NEVER passes by another open valve, so it coincides with the original graph `G`.
- We must always move in between open valves to increase the flow, hence we filter our distance matrix on open valves only
- It's always best to take the shortest path to open your next valve

In [32]:
# D = (pd.DataFrame(dict(nx.all_pairs_dijkstra_path_length(G_tf)))
# #      .filter(like='o', axis='rows')
# #      .filter(like='o', axis='columns')
#     )
# D
# doesn't yield the expected result for some reason...
from itertools import islice

def k_shortest_paths(G, source, target, k, weight=None):
    return list(
        islice(nx.shortest_simple_paths(G, source, target, weight=weight), k)
    )

def get_distance_matrix_and_simple_paths(G, k=3):
    open_valves = [v for v in G if v.endswith('o')]
    combos = filter(lambda t: t[0] != t[1], product(list(G), open_valves))

    all_simple_paths = {(s, d): k_shortest_paths(G, s, d, k=k)
                          for s, d in combos}
    all_min_distances = [(s, d, min([len(p) - 1 for p in path_list])) for (s, d), path_list in all_simple_paths.items()]

    D = pd.DataFrame(all_min_distances).set_index([0, 1]).unstack().fillna(0).astype(int)
    D.columns = D.columns.levels[1]
    
    SP = {(s, d): list(filter(lambda p: len(p) - 1 == D.loc[s, d], l)) for (s, d), l in all_simple_paths.items()}
    
    return D, SP

D, SP = get_distance_matrix_and_simple_paths(G_tf)
D

1,BBo,CCo,DDo,EEo,HHo,JJo
0,,,,,,
AA,2,3,2,3,6,3
BB,1,2,3,4,7,4
BBo,0,2,3,4,7,4
CC,2,1,2,3,6,5
CCo,2,0,2,3,6,5
DD,3,2,1,2,5,4
DDo,3,2,0,2,5,4
EE,4,3,2,1,4,5
EEo,4,3,2,0,4,5


We also want to have the shortest paths for these ready for navigation! (without recalculating)

In [33]:
SP

{('AA', 'BBo'): [['AA', 'BB', 'BBo']],
 ('AA', 'CCo'): [['AA', 'BB', 'CC', 'CCo'], ['AA', 'DD', 'CC', 'CCo']],
 ('AA', 'DDo'): [['AA', 'DD', 'DDo']],
 ('AA', 'EEo'): [['AA', 'DD', 'EE', 'EEo']],
 ('AA', 'HHo'): [['AA', 'DD', 'EE', 'FF', 'GG', 'HH', 'HHo']],
 ('AA', 'JJo'): [['AA', 'II', 'JJ', 'JJo']],
 ('BB', 'BBo'): [['BB', 'BBo']],
 ('BB', 'CCo'): [['BB', 'CC', 'CCo']],
 ('BB', 'DDo'): [['BB', 'AA', 'DD', 'DDo'], ['BB', 'CC', 'DD', 'DDo']],
 ('BB', 'EEo'): [['BB', 'CC', 'DD', 'EE', 'EEo'],
  ['BB', 'AA', 'DD', 'EE', 'EEo']],
 ('BB', 'HHo'): [['BB', 'AA', 'DD', 'EE', 'FF', 'GG', 'HH', 'HHo'],
  ['BB', 'CC', 'DD', 'EE', 'FF', 'GG', 'HH', 'HHo']],
 ('BB', 'JJo'): [['BB', 'AA', 'II', 'JJ', 'JJo']],
 ('CC', 'BBo'): [['CC', 'BB', 'BBo']],
 ('CC', 'CCo'): [['CC', 'CCo']],
 ('CC', 'DDo'): [['CC', 'DD', 'DDo']],
 ('CC', 'EEo'): [['CC', 'DD', 'EE', 'EEo']],
 ('CC', 'HHo'): [['CC', 'DD', 'EE', 'FF', 'GG', 'HH', 'HHo']],
 ('CC', 'JJo'): [['CC', 'BB', 'AA', 'II', 'JJ', 'JJo'],
  ['CC', 'DD', 'AA'

### 5. Traversing path
Whenever we have chosen a path with opening a valve, we need to remove the vertices we've created (since the valve cannot be opened again to increase the flow). This is equivalent to removing the source node (first node in the path).

In [34]:
def traverse_graph(G, path):
    # take a copy, since we will want to keep the original
    if not path[0].endswith('o'):
        return G
    G_update = G.copy()
    G_update.remove_node(path[0])
    return G_update

### 6. Finding closest open valves
Find the top $K$ closest valves that are open.

In [35]:
def find_K_closest_open_valves(G, D, source, K=3, exclude=[], max_distance=None):
    remaining = [v for v in G if v.endswith('o') and v != source and v not in exclude]
    top_valves = D.loc[source, remaining].sort_values()
    if max_distance is not None:
        top_valves = top_valves[top_valves <= max_distance]
    topK_valves = top_valves.head(K)
    return topK_valves.index.tolist()

source = 'START'
K = 10
destinations = find_K_closest_open_valves(G_tf, D, source, K, max_distance=3)
destinations

['BBo', 'DDo']

Now we also want to know which paths are involved of course!!

In [36]:
def get_shortest_simple_paths(SP, source, destinations):
    return reduce(lambda x, y: x + y, [SP[(source, d)] for d in destinations])


get_shortest_simple_paths(SP, source, destinations)

[['START', 'AA', 'BB', 'BBo'], ['START', 'AA', 'DD', 'DDo']]

### 7. Recursively finding the optimal trajectory

In [37]:
def find_max_pressure_trajectory(G, D, SP, source, visited=[], time=30, path=[], flow=0, 
                                 pressure=0, K=6, depth=20, memory=None):        
    if len(path) == 0:
        path = [source]
    
    if time <= 0 or depth <= 0:
        return path, pressure, flow
    
    
    visited = visited + [source]
    destinations = find_K_closest_open_valves(G, D, source, exclude=visited, K=K, max_distance=time)
    if len(destinations) == 0:
        return path, pressure + time * flow, flow
    
    best = path, pressure, flow 

    possible_paths = get_shortest_simple_paths(SP, source, destinations)
    for possible_path in possible_paths:
        extra_pressure, extra_flow, extra_time = calculate_pressure(G, possible_path, memory)
        
        p = pressure + extra_time * flow + extra_pressure
        f = flow + extra_flow
        t = time - extra_time
        new_path = path + possible_path[1:]
        new_source = possible_path[-1]

        final_path, final_pressure, final_flow = \
            find_max_pressure_trajectory(G, D, SP, new_source, visited, t, new_path, f, p, K, depth-1, memory)
        
        if final_pressure > best[1]:
            best = final_path, final_pressure, final_flow
    
    return best

In [38]:
M = {}
find_max_pressure_trajectory(G_tf, D, SP, 'START', time=30, K=3, depth=10, path=[], memory=M)
# get_shortest_simple_paths(G_tf, 'AA', ['BBo', 'DDo', 'CCo', 'EEo', 'JJo'])

(['START',
  'AA',
  'DD',
  'DDo',
  'AA',
  'BB',
  'BBo',
  'AA',
  'II',
  'JJ',
  'JJo',
  'II',
  'AA',
  'DD',
  'EE',
  'FF',
  'GG',
  'HH',
  'HHo',
  'GG',
  'FF',
  'EE',
  'EEo',
  'DD',
  'CC',
  'CCo'],
 1651,
 81)

### 8. Solution for the input

In [39]:
G = read_input(INPUT_PATH)
network = nx_to_network(G, directed=True)
network.show("test_input.html")

In [40]:
G_tf = transform(G, start='AA')
D, SP = get_distance_matrix_and_simple_paths(G_tf)

# wihtout cache
%time find_max_pressure_trajectory(G_tf, D, SP, 'START', time=30, K=5, depth=15, path=[])

CPU times: user 10 s, sys: 24.1 ms, total: 10.1 s
Wall time: 10.1 s


(['START',
  'AA',
  'RO',
  'TC',
  'KU',
  'KUo',
  'TF',
  'FR',
  'FRo',
  'ZK',
  'ER',
  'QO',
  'QOo',
  'MF',
  'GQ',
  'VD',
  'VDo',
  'TR',
  'AJ',
  'AJo',
  'QX',
  'CG',
  'CGo',
  'FF',
  'ZL',
  'WI',
  'WIo',
  'ZJ',
  'GJ',
  'GJo'],
 1792,
 128)

In [41]:
# with cache
M = {}
%time find_max_pressure_trajectory(G_tf, D, SP, 'START', time=30, K=5, depth=15, path=[], memory=M)

CPU times: user 9.73 s, sys: 31.3 ms, total: 9.76 s
Wall time: 9.83 s


(['START',
  'AA',
  'RO',
  'TC',
  'KU',
  'KUo',
  'TF',
  'FR',
  'FRo',
  'ZK',
  'ER',
  'QO',
  'QOo',
  'MF',
  'GQ',
  'VD',
  'VDo',
  'TR',
  'AJ',
  'AJo',
  'QX',
  'CG',
  'CGo',
  'FF',
  'ZL',
  'WI',
  'WIo',
  'ZJ',
  'GJ',
  'GJo'],
 1792,
 128)

## Part 2

Two players at the same time...

**Key observations:**
- fully symmetrical, which path the elf or the elephant takes doesn't matter
- only use 1 shortest path
- try to optimise speed with a cache
- instead of always removing the nodes, keep track with a list of nodes visited, since they dont have any impact anyway on the shortest distances.

In [45]:
def find_max_pressure_trajectory_2p(G, D, SP, source1='START', source2='START', visited=[],
                                    time1=26, time2=26, path1=['START'], path2=['START'],
                                    flow1=0, flow2=0, pressure1=0, pressure2=0, N=[], K=6, depth=20, 
                                    memory=memory):        
    if (time1 <= 0 and time2 <= 0) or depth <= 0:
        if depth <= 0:
            print(f'Ran out of depth: time1: {time1}, time2: {time2}')
        return path1, path2, pressure1 + pressure2, flow1 + flow2        
    
    visited = visited + list(set([source1, source2]))

    destinations1 = find_K_closest_open_valves(G, D, source1, exclude=visited, K=K, max_distance=time1)
    destinations2 = find_K_closest_open_valves(G, D, source2, exclude=visited, K=K, max_distance=time2)
    
    if len(destinations1) == 0 and len(destinations2) == 0:
        return path1, path2, pressure1 + pressure2 + time1 * flow1 + time2 * flow2, flow1 + flow2
    if len(destinations1) == 0:
        # Only the second player continues
        path2, pressure2, flow2 = find_max_pressure_trajectory(G, D, SP, source2, visited,
                                                               time=time2, path=path2, flow=flow2, pressure=pressure2, 
                                                               depth=depth-1, memory=memory)
        return path1, path2, pressure1 + time1 * flow1 + pressure2, flow1 + flow2
    if len(destinations2) == 0:
        # Only the first player continues
        path1, pressure1, flow1 = find_max_pressure_trajectory(G, D, SP, source1, visited,
                                                               time=time1, path=path1, flow=flow1, pressure=pressure1,
                                                               depth=depth-1, memory=memory)
        return path1, path2, pressure1 + pressure2 + time2 * flow2, flow1 + flow2
    
    
    best = path1, path2, pressure1 + pressure2, flow1 + flow2
    # doesn't make sense for both the elf and the elephant to go to the same destination
    # if they're at the same location, we also don't care which elf/eleph goes where
    # they should not be allowed to travel to each other's location as well!
    if source1 == source2:
        combos = permutations(destinations1, 2)
    else:
        combos = filter(lambda t: t[0] != t[1] and t[0] != source2 and t[1] != source1, 
                        product(destinations1, destinations2))

    for d1, d2 in combos:
        possible_path1 = SP[(source1, d1)][0]
        possible_path2 = SP[(source2, d2)][0]
        
        extra_pressure1, extra_flow1, extra_time1 = calculate_pressure(G, possible_path1, memory)
        extra_pressure2, extra_flow2, extra_time2 = calculate_pressure(G, possible_path2, memory)
        
        p1, p2 = pressure1 + extra_time1 * flow1 + extra_pressure1, pressure2 + extra_time2 * flow2 + extra_pressure2
        f1, f2 = flow1 + extra_flow1, flow2 + extra_flow2
        t1, t2 = time1 - extra_time1, time2 - extra_time2
        
        new_path1, new_path2 = path1 + possible_path1[1:], path2 + possible_path2[1:]
        new_source1, new_source2 = possible_path1[-1], possible_path2[-1]

        final_path1, final_path2, final_pressure, final_flow = \
            find_max_pressure_trajectory_2p(G, D, SP, new_source1, new_source2, 
                                             visited,
                                             t1, t2, new_path1, new_path2,
                                             f1, f2, p1, p2, K, depth-1, memory=memory)
        
        if final_pressure > best[2]:
            best = final_path1, final_path2, final_pressure, final_flow
    
    return best

### Test case

In [46]:
G = read_input(TEST_PATH)
G_tf = transform(G, start='AA')
D, SP = get_distance_matrix_and_simple_paths(G_tf)

%time find_max_pressure_trajectory_2p(G_tf, D, SP, time1=26, time2=26, K=5, depth=15)

CPU times: user 346 ms, sys: 1.97 ms, total: 347 ms
Wall time: 349 ms


(['START',
  'AA',
  'DD',
  'DDo',
  'EE',
  'FF',
  'GG',
  'HH',
  'HHo',
  'GG',
  'FF',
  'EE',
  'EEo'],
 ['START', 'AA', 'II', 'JJ', 'JJo', 'II', 'AA', 'BB', 'BBo', 'CC', 'CCo'],
 1707,
 81)

### Input case

In [72]:
G = read_input(INPUT_PATH)
G_tf = transform(G, start='AA')
D, SP = get_distance_matrix_and_simple_paths(G_tf)

memory = dict()
%time p1, p2, p, f = find_max_pressure_trajectory_2p(G_tf, D, SP, time1=15, time2=15, K=3, depth=15, memory=memory)
p1, p2, p, f

CPU times: user 5.23 s, sys: 10.8 ms, total: 5.24 s
Wall time: 5.24 s


(['START',
  'AA',
  'VQ',
  'PI',
  'PIo',
  'EU',
  'GJ',
  'GJo',
  'YJ',
  'EQ',
  'RC',
  'RCo',
  'WR',
  'TX',
  'FV',
  'FVo'],
 ['START',
  'AA',
  'DZ',
  'VO',
  'VD',
  'VDo',
  'GQ',
  'MF',
  'QO',
  'QOo',
  'ER',
  'ZK',
  'FR',
  'FRo'],
 771,
 129)

In [66]:
from itertools import pairwise

for v1, v2 in pairwise(p1):
    G_tf.edges[(v1, v2)]['color'] = 'red'
    
for v1, v2 in pairwise(p2):
    G_tf.edges[(v1, v2)]['color'] = 'green'
    
network = nx_to_network(G_tf, directed=True)
network.show('part2.html')

It takes a long time to complete, apparently (by looking with `cProfile`) the code can still be optimised within `find_K_closest_open_valves` as the code spends most of it's time there.

In [94]:
%time p1, p2, p, f = find_max_pressure_trajectory_2p(G_tf, D, SP, time1=26, time2=26, K=3, depth=15, memory=memory)
p1, p2, p, f

CPU times: user 1h 1min 57s, sys: 22.1 s, total: 1h 2min 20s
Wall time: 1h 2min 35s


(['START',
  'AA',
  'VQ',
  'PI',
  'PIo',
  'EU',
  'GJ',
  'GJo',
  'YJ',
  'EQ',
  'RC',
  'RCo',
  'WR',
  'TX',
  'FV',
  'FVo',
  'KV',
  'OF',
  'OFo'],
 ['START',
  'AA',
  'DZ',
  'VO',
  'VD',
  'VDo',
  'GQ',
  'MF',
  'QO',
  'QOo',
  'ER',
  'ZK',
  'FR',
  'FRo',
  'TF',
  'KU',
  'KUo',
  'YW',
  'MK',
  'AJ',
  'AJo',
  'QX',
  'CG',
  'CGo',
  'SU',
  'LM',
  'LMo'],
 2587,
 182)

In [75]:
import cProfile
cProfile.run("find_max_pressure_trajectory_2p(G_tf, D, SP, time1=20, time2=20, K=3, depth=15, memory=memory)")

         908222597 function calls (898334195 primitive calls) in 263.461 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   524534    0.173    0.000    0.173    0.000 1926699158.py:3(calculate_pressure)
 183253/1    1.350    0.000  263.460  263.460 2155837623.py:1(find_max_pressure_trajectory_2p)
   205608    0.054    0.000    0.054    0.000 2155837623.py:35(<lambda>)
    73402    0.054    0.000    0.140    0.000 3266533892.py:1(get_shortest_simple_paths)
    75384    0.013    0.000    0.013    0.000 3266533892.py:2(<lambda>)
    73402    0.047    0.000    0.047    0.000 3266533892.py:2(<listcomp>)
217040/59010    0.564    0.000   79.007    0.001 4156550391.py:1(find_max_pressure_trajectory)
   477356    2.374    0.000  261.138    0.001 978636205.py:1(find_K_closest_open_valves)
   477356    3.373    0.000    5.928    0.000 978636205.py:2(<listcomp>)
   477356    0.207    0.000    0.745    0.000 <__array_function__ internal